In [1]:
import sys
from PyQt5.QtWidgets import *

class MyWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("PyEmdeddedComputing")
        self.setGeometry(300,300,300,400)
if __name__ == "__main__":
    app = QApplication(sys.argv)
    mywindow = MyWindow()
    mywindow.show()
    app.exec_()

In [1]:
import sys
from PyQt5.QtCore import Qt
from PyQt5.QtWidgets import (QWidget, QLCDNumber,QSlider,QVBoxLayout,QApplication)

class Example(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
    def initUI(self):
        lcd = QLCDNumber(self)
        sld = QSlider(Qt.Horizontal,self)
        vbox = QVBoxLayout()
        vbox.addWidget(lcd)
        vbox.addWidget(sld)
        self.setLayout(vbox)
        sld.valueChanged.connect(lcd.display)
        self.setGeometry(300,300,250,150)
        self.setWindowTitle('signal and slot')
        
if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = Example()
    ex.show()
    app.exec_()

# 얼굴인식

In [1]:
from flask import Flask,request
from werkzeug import secure_filename
import cv2
import numpy as np
import os
import threading
import requests

#얼굴 인식
def update(recognizer):
    URL = "http://172.20.10.4:8000/get_data"
    res = requests.get(URL)
    open('trainer.yml','wb').write(res.content)
    print(type(res))
    recognizer.read("trainer.yml")
    threading.Timer(10, update).start()

#버튼을 눌렀을 때 실행할 함수
def recognize_face():
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read('trainer.yml')
    #update(recognizer)
    cascadePath = "haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(cascadePath);
    font = cv2.FONT_HERSHEY_SIMPLEX
    #iniciate id counter
    id = 0

    #SQL에서 이름 목록 가지고 옴
    names = ['??' ,'SeongHeon','SeongHeon','SeongHeon']
    # Initialize and start realtime video capture
    cam = cv2.VideoCapture(0)
    cam.set(3, 640) # set video widht
    cam.set(4, 480) # set video height

    # Define min window size to be recognized as a face
    minW = 0.1*cam.get(3)
    minH = 0.1*cam.get(4)
    while True:
        ret, img =cam.read()
        img = cv2.flip(img, 1) # flip video image vertically
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
        faces = faceCascade.detectMultiScale( 
            gray,
            scaleFactor = 1.2,
            minNeighbors = 5,
            minSize = (int(minW), int(minH)),
           )

        for(x,y,w,h) in faces:
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
            id, confidence = recognizer.predict(gray[y:y+h,x:x+w])
            # Check if confidence is less them 100 ==> "0" is perfect match
            if (confidence < 100):
                name = names[id]
                confidence = "  {0}%".format(round(100 - confidence))
                print("\n [INFO] Exiting Program and cleanup stuff")
                cv2.VideoCapture(0).release()
                cv2.destroyAllWindows()
                return id;
            else:
                id = "unknown"
                confidence = "  {0}%".format(round(100 - confidence))
                #인식 실패, 일반 Interface 제공
        
            cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
            cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)  
    
        cv2.imshow('camera',img) 
        k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
        if k == 27:
            break

    # Do a bit of cleanup
    print("\n [INFO] Exiting Program and cleanup stuff")
    cv2.VideoCapture(0).release()
    cv2.destroyAllWindows()
#print(recognize_face())

# PyQt GUI

In [1]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtCore import Qt
from PyQt5.uic import loadUi
import pymysql
from PyQt5.QtGui import *

import pandas as pd #데이터 읽어야 됨
from flask import Flask,request
from werkzeug import secure_filename
import cv2
import numpy as np
import os
import threading
import requests

#얼굴 인식

gukbab = ['순대국밥','수육국밥']
gimbab = ['참치김밥','야채김밥']
jjigae = ['김치찌개','된장찌개']
hambug = ['치즈버거','불고기버거']
pizza = ['고구마피자','포테이토피자']
ramen = ['진라면','신라면']
salad = ['연어샐러드','단호박샐러드']
meat = ['목살구이','삼겹살구이']
bread = ['통밀빵','단팥빵']
sandwich = ['햄에그샌드위치','야채샌드위치']

def update(recognizer):
    URL = "http://172.20.10.4:8000/get_data"
    res = requests.get(URL)
    open('trainer.yml','wb').write(res.content)
    print(type(res))
    recognizer.read("trainer.yml")
    threading.Timer(10, update).start()

#버튼을 눌렀을 때 실행할 함수
def recognize_face():
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read('trainer.yml')
    #update(recognizer)
    cascadePath = "haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(cascadePath);
    font = cv2.FONT_HERSHEY_SIMPLEX
    #iniciate id counter
    id = 0
    
    #SQL에서 이름 목록 가지고 옴
    names = ['??' ,'SeongHeon','SeongHeon','SeongHeon']
    # Initialize and start realtime video capture
    cam = cv2.VideoCapture(0)
    cam.set(3, 640) # set video widht
    cam.set(4, 480) # set video height

    # Define min window size to be recognized as a face
    minW = 0.1*cam.get(3)
    minH = 0.1*cam.get(4)
    count = 0
    while True:
        count = count + 1;
        ret, img =cam.read()
        img = cv2.flip(img, 1) # flip video image vertically
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
        faces = faceCascade.detectMultiScale( 
            gray,
            scaleFactor = 1.2,
            minNeighbors = 5,
            minSize = (int(minW), int(minH)),
           )

        for(x,y,w,h) in faces:
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
            id, confidence = recognizer.predict(gray[y:y+h,x:x+w])
            # Check if confidence is less them 100 ==> "0" is perfect match
            if (confidence < 100):
                name = names[id]
                confidence = "  {0}%".format(round(100 - confidence))
                print("\n [INFO] Exiting Program and cleanup stuff")
                
                if count >= 150:
                    cv2.VideoCapture(0).release()
                    cv2.destroyAllWindows()
                    return id;
            else:
                id = "unknown"
                confidence = "  {0}%".format(round(100 - confidence))
                #인식 실패, 일반 Interface 제공
        
            cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
            cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)  
    
        cv2.imshow('camera',img) 
        k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
        if k == 27:
            break

    # Do a bit of cleanup
    print("\n [INFO] Exiting Program and cleanup stuff")
    cv2.VideoCapture(0).release()
    cv2.destroyAllWindows()
#print(recognize_face())

def select_sql(id):
    ip = 'localhost'
    conn = pymysql.connect(host = ip ,user='root',password='tjdgjs789',db='ku_smart_kiosk',charset='utf8')
    try:
        cursor = conn.cursor(pymysql.cursors.DictCursor)
        cursor.execute('''select * from user where id ='''+str(id))
        conn.commit()
        return cursor.fetchall()
    finally:
        conn.close()

def set_ui_std(ui,result):
    ui.pushButton.hide()
    ui.pushButton.show()
    ui.tabWidget.show()
    ui.name_text.setText("안녕하세요  "+ result[0]['name'] + " 님")
    ui.name_text.show()
    if result[0]['love'] == '국밥':
            set_love(ui.image_label,gukbab[0]+'.jpg',ui.pushButton_2,gukbab[0])
            set_love(ui.image_label2,gukbab[1]+'.jpg',ui.pushButton_3,gukbab[1])
    if result[0]['love'] == '김밥':
            set_love(ui.image_label,gukbab[0]+'.jpg',ui.pushButton_2,gimbab[0])
            set_love(ui.image_label2,gukbab[1]+'.jpg',ui.pushButton_3,gimbab[1])
            
    if result[0]['love'] == '찌개':
            set_love(ui.image_label,gukbab[0]+'.jpg',ui.pushButton_2,jjigae[0])
            set_love(ui.image_label2,gukbab[1]+'.jpg',ui.pushButton_3,jjigae[1])
            
    if result[0]['love'] == '햄버거':
            set_love(ui.image_label,gukbab[0]+'.jpg',ui.pushButton_2,hambug[0])
            set_love(ui.image_label2,gukbab[1]+'.jpg',ui.pushButton_3,hambug[1])
            
    if result[0]['love'] == '피자':
            set_love(ui.image_label,gukbab[0]+'.jpg',ui.pushButton_2,pizza[0])
            set_love(ui.image_label2,gukbab[1]+'.jpg',ui.pushButton_3,pizza[1])
            
    if result[0]['love'] == '라면/우동':
            set_love(ui.image_label,gukbab[0]+'.jpg',ui.pushButton_2,ramen[0])
            set_love(ui.image_label2,gukbab[1]+'.jpg',ui.pushButton_3,ramen[1])
            
    if result[0]['love'] == '샐러드':
            set_love(ui.image_label,gukbab[0]+'.jpg',ui.pushButton_2,salad[0])
            set_love(ui.image_label2,gukbab[1]+'.jpg',ui.pushButton_3,salad[1])
    if result[0]['love'] == '고기':
            set_love(ui.image_label,gukbab[0]+'.jpg',ui.pushButton_2,meat[0])
            set_love(ui.image_label2,gukbab[1]+'.jpg',ui.pushButton_3,meat[1])
            
    if result[0]['love'] == '빵':
            set_love(ui.image_label,gukbab[0]+'.jpg',ui.pushButton_2,bread[0])
            set_love(ui.image_label2,gukbab[1]+'.jpg',ui.pushButton_3,bread[1])
            
    if result[0]['love'] == '샌드위치':
            set_love(ui.image_label,gukbab[0]+'.jpg',ui.pushButton_2,sandwich[0])
            set_love(ui.image_label2,gukbab[1]+'.jpg',ui.pushButton_3,sandwich[1])
            
    

def set_love(label,image,text_label,item):
        pixmap = QPixmap(image)
        pixmap = pixmap.scaledToWidth(200)
        pixmap = pixmap.scaledToHeight(200)
        label.setPixmap(QPixmap(pixmap))
        text_label.setText(item)
        
        if result[0]['love'] == '국밥':
            set_love(ui.image_label,gukbab[0]+'.jpg',ui.pushButton_2,gukbab[0])
            set_love(ui.image_label2,gukbab[1]+'.jpg',ui.pushButton_3,gukbab[1])
        if result[0]['love'] == '김밥':
            set_love(ui.image_label,gukbab[0]+'.jpg',ui.pushButton_2,gimbab[0])
            set_love(ui.image_label2,gukbab[1]+'.jpg',ui.pushButton_3,gimbab[1])
            
        if result[0]['love'] == '찌개':
            set_love(ui.image_label,gukbab[0]+'.jpg',ui.pushButton_2,jjigae[0])
            set_love(ui.image_label2,gukbab[1]+'.jpg',ui.pushButton_3,jjigae[1])
            
        if result[0]['love'] == '햄버거':
            set_love(ui.image_label,gukbab[0]+'.jpg',ui.pushButton_2,hambug[0])
            set_love(ui.image_label2,gukbab[1]+'.jpg',ui.pushButton_3,hambug[1])
            
        if result[0]['love'] == '피자':
            set_love(ui.image_label,gukbab[0]+'.jpg',ui.pushButton_2,pizza[0])
            set_love(ui.image_label2,gukbab[1]+'.jpg',ui.pushButton_3,pizza[1])
            
        if result[0]['love'] == '라면/우동':
            set_love(ui.image_label,gukbab[0]+'.jpg',ui.pushButton_2,ramen[0])
            set_love(ui.image_label2,gukbab[1]+'.jpg',ui.pushButton_3,ramen[1])
            
        if result[0]['love'] == '샐러드':
            set_love(ui.image_label,gukbab[0]+'.jpg',ui.pushButton_2,salad[0])
            set_love(ui.image_label2,gukbab[1]+'.jpg',ui.pushButton_3,salad[1])
        if result[0]['love'] == '고기':
            set_love(ui.image_label,gukbab[0]+'.jpg',ui.pushButton_2,meat[0])
            set_love(ui.image_label2,gukbab[1]+'.jpg',ui.pushButton_3,meat[1])
            
        if result[0]['love'] == '빵':
            set_love(ui.image_label,gukbab[0]+'.jpg',ui.pushButton_2,bread[0])
            set_love(ui.image_label2,gukbab[1]+'.jpg',ui.pushButton_3,bread[1])
            
        if result[0]['love'] == '샌드위치':
            set_love(ui.image_label,gukbab[0]+'.jpg',ui.pushButton_2,sandwich[0])
            set_love(ui.image_label2,gukbab[1]+'.jpg',ui.pushButton_3,sandwich[1])
        
    
def set_ui_blind(ui,result):
    ui.pushButton.hide()
    ui.pushButton.show()
    ui.tabWidget.show()
    ui.name_text.setText("안녕하세요  "+ result[0]['name'] + " 님")
    ui.name_text.show()

class MyWindow(QMainWindow):
    def __init__(self,*args):
        super(self.__class__,self).__init__(*args)
        self.ui = loadUi('C:\\Users\\rlat3\\임베디드 컴퓨팅\\button.ui',self)
        self.ui.pushButton.clicked.connect(self.btn1_clicked)
    def btn1_clicked(self):
        id = recognize_face()
        result = select_sql(id)
        if result[0]['blind'] =="예":
            self.ui = loadUi('C:\\Users\\rlat3\\임베디드 컴퓨팅\\kiosk_blind.ui',self)
            set_ui_blind(self.ui,result)
            
        else:
            self.ui = loadUi('C:\\Users\\rlat3\\임베디드 컴퓨팅\\kiosk.ui',self)
            set_ui_std(self.ui,result)



if __name__ == "__main__":
    app = QApplication(sys.argv)
    mywindow = MyWindow()
    mywindow.show()
    sys.exit(app.exec_())
        


 [INFO] Exiting Program and cleanup stuff

 [INFO] Exiting Program and cleanup stuff

 [INFO] Exiting Program and cleanup stuff

 [INFO] Exiting Program and cleanup stuff

 [INFO] Exiting Program and cleanup stuff

 [INFO] Exiting Program and cleanup stuff

 [INFO] Exiting Program and cleanup stuff

 [INFO] Exiting Program and cleanup stuff

 [INFO] Exiting Program and cleanup stuff

 [INFO] Exiting Program and cleanup stuff

 [INFO] Exiting Program and cleanup stuff

 [INFO] Exiting Program and cleanup stuff

 [INFO] Exiting Program and cleanup stuff

 [INFO] Exiting Program and cleanup stuff

 [INFO] Exiting Program and cleanup stuff

 [INFO] Exiting Program and cleanup stuff

 [INFO] Exiting Program and cleanup stuff

 [INFO] Exiting Program and cleanup stuff

 [INFO] Exiting Program and cleanup stuff

 [INFO] Exiting Program and cleanup stuff

 [INFO] Exiting Program and cleanup stuff

 [INFO] Exiting Program and cleanup stuff

 [INFO] Exiting Program and cleanup stuff

 [INFO] Ex

SystemExit: 0

C:\Users\rlat3\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
